In [77]:
import numpy as np
import pandas as pd

from dataqualitypipeline import DQPipeline
from dataqualitypipeline import initialize_autoencoder, initialize_autoencoder_modified

In [78]:
df_data = pd.read_csv("./HOWTO/players_20.csv")
df_data.head(1)

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,...,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left,5,4,4,Medium/Low,...,93+2,93+2,93+2,93+2,93+2,92+2,87+2,87+2,87+2,92+2,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2


# FIFA World Cup Player Ratings
---
https://www.kaggle.com/datasets/thedevastator/fifa-world-cup-anomaly-detection-in-player-ratin

### Unsupervised Anomaly Detection on (train) data

In [79]:
only_columns = df_data.columns.to_list()
# only_columns[:35]

In [88]:
from pyod.models.iforest import IForest
from pyod.models.lof import LOF
# import torch
clf_if = IForest(n_jobs=-1)
clf_ae = initialize_autoencoder_modified(epochs=100)
clf_lof = LOF(n_jobs=-1)

dq_pipe = DQPipeline(
    nominal_columns=["player_tags","preferred_foot",
                     "work_rate","team_position","loaned_from"],

    exclude_columns=["player_url","body_type","short_name", "long_name", 
                     "team_jersey_number","joined","contract_valid_until",
                     "real_face","nation_position","player_positions","nationality","club"],

    time_column_names=["dob"],
    deactivate_pattern_recognition=True,
    remove_columns_with_no_variance=True,
)

X_output = dq_pipe.run_pipeline(
    # X_train=df_data.iloc[:,0:35],
    X_train=df_data,
    clf=clf_if,
    dump_model=False,
)



Using cpu device
Batch size: 8192
15 cores will be used...
Only X_train input will be transformed...
Running only Transformation-Pipeline...
No Variance in follow Train Columns:  Index(['Preprocessing Pipeline__Datetime__timeseries__X__X__dob_HOUR',
       'Preprocessing Pipeline__Datetime__timeseries__X__X__dob_MINUTE',
       'Preprocessing Pipeline__Datetime__timeseries__X__X__dob_SECOND',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_sofifa_id',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_player_url',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_short_name',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_long_name',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_age',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_dob',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_height_cm',
       ...
       'Categorical_PatternExtraction__pattern_processing_inner_

ValueError: Input contains NaN.

In [85]:
# dq_pipe.X_train_transformed

In [86]:
pd.set_option('display.max_columns', 40)

In [87]:
X_output.head(20)

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,AnomalyScore,MAD_Total,Tukey_Total
10693,156321,https://sofifa.com/player/156321/adebayo-akinf...,A. Akinfenwa,Adebayo Akinfenwa,37,1982-05-10,178,110,England,Wycombe Wanderers,65,65,190000,2000,ST,Right,1,3,2,Low/Low,Akinfenwa,Yes,333000.0,#Strength,ST,20.0,NaN,2016-07-10,2020.0,NaN,NaN,43.0,63.0,54.0,56.0,1.000000,1,3
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,92,92,105500000,290000,"LW, CAM",Right,5,5,5,High/Medium,Neymar,Yes,195200000.0,"#Speedster, #Dribbler, #Playmaker , #Crosser,...",CAM,10.0,NaN,2017-08-03,2022.0,LW,10.0,91.0,85.0,87.0,95.0,0.277577,9,11
37,188350,https://sofifa.com/player/188350/marco-reus/20...,M. Reus,Marco Reus,30,1989-05-31,180,71,Germany,Borussia Dortmund,88,88,56000000,170000,"CAM, LM, ST",Right,4,4,4,High/Medium,Lean,Yes,92400000.0,"#Dribbler, #Playmaker , #Distance Shooter, #F...",CAM,11.0,NaN,2012-07-01,2023.0,ST,11.0,85.0,88.0,84.0,87.0,0.271090,6,9
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left,5,4,4,Medium/Low,Messi,Yes,195800000.0,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...",RW,10.0,NaN,2004-07-01,2021.0,NaN,NaN,87.0,92.0,92.0,96.0,0.222083,9,11
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,93,93,58500000,405000,"ST, LW",Right,5,4,5,High/Low,C. Ronaldo,Yes,96500000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",LW,7.0,NaN,2018-07-10,2022.0,LS,7.0,90.0,93.0,82.0,89.0,0.211646,9,11
1612,251691,https://sofifa.com/player/251691/alan-zamorado...,A. Zamorado,Alan Zamorado,38,1980-12-17,182,78,Ecuador,Ecuador,76,76,0,0,LM,Left,2,3,3,Medium/Low,Lean,No,NaN,"#Distance Shooter, #FK Specialist",NaN,NaN,NaN,NaN,NaN,LM,17.0,57.0,78.0,81.0,77.0,0.191686,0,1
9,209331,https://sofifa.com/player/209331/mohamed-salah...,M. Salah,Mohamed Salah Ghaly,27,1992-06-15,175,71,Egypt,Liverpool,90,90,80500000,240000,"RW, ST",Left,3,3,4,High/Medium,PLAYER_BODY_TYPE_25,Yes,148900000.0,"#Speedster, #Dribbler, #Acrobat, #Clinical Fin...",RW,11.0,NaN,2017-07-01,2023.0,RW,10.0,93.0,86.0,81.0,89.0,0.191245,5,10
96,41236,https://sofifa.com/player/41236/zlatan-ibrahim...,Z. Ibrahimović,Zlatan Ibrahimović,37,1981-10-03,195,95,Sweden,LA Galaxy,85,85,14000000,15000,ST,Right,5,4,5,Medium/Low,Normal,Yes,21000000.0,"#Poacher, #Aerial Threat, #Distance Shooter, #...",ST,9.0,NaN,2018-03-23,2019.0,NaN,NaN,56.0,88.0,77.0,77.0,0.189089,6,9
85,212198,https://sofifa.com/player/212198/bruno-miguel-...,Bruno Fernandes,Bruno Miguel Borges Fernandes,24,1994-09-08,179,69,Portugal,Sporting CP,85,88,49000000,24000,"CAM, CM",Right,2,4,4,High/High,Lean,No,101700000.0,"#Playmaker , #Engine, #Distance Shooter, #Cro...",RCM,8.0,NaN,2017-07-01,2023.0,RCM,16.0,77.0,83.0,87.0,85.0,0.180515,6,7
307,189068,https://sofifa.com/player/189068/bas-dost/20/1...,B. Dost,Bas Dost,30,1989-05-31,196,89,Netherlands,Sporting CP,82,82,21500000,23000,ST,Right,3,3,2,Medium/Medium,Lean,No,43000000.0,"#Poacher, #Aerial Threat, #Strength, #Complete...",SUB,28.0,NaN,2016-08-28,2021.0,NaN,NaN,50.0,82.0,57.0,66.0,0.177296,3,5


In [95]:
df_data[["nation_jersey_number"]]

,nation_jersey_number
0,NaN
1,7.0
2,10.0
3,1.0
4,10.0
...,...
18273,NaN
18274,NaN
18275,NaN
18276,NaN


In [96]:
df_data[["nation_jersey_number"]].isna().sum()

nation_jersey_number    17152
dtype: int64